In [2]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
     --- ------------------------------------ 0.2/2.4 MB 6.9 MB/s eta 0:00:01
     ------------ --------------------------- 0.8/2.4 MB 9.6 MB/s eta 0:00:01
     --------------------- ------------------ 1.3/2.4 MB 10.3 MB/s eta 0:00:01
     ------------------------------ --------- 1.8/2.4 MB 10.7 MB/s eta 0:00:01
     ---------------------------------------  2.4/2.4 MB 10.8 MB/s eta 0:00:01
     ---------------------------------------- 2.4/2.4 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/113.5 kB ? eta -:--:--
     -------------------------------------- 113.5/113.5 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.0 MB 2.0 MB/s eta 0:00:01
     ---------------------- ----------------- 0.6/1.0 MB 7.0 MB/s eta 0:00:01
     -------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dotenv import load_dotenv

load_dotenv()



True

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729398312, 'localtime': '2024-10-19 21:25'}, 'current': {'last_updated_epoch': 1729397700, 'last_updated': '2024-10-19 21:15', 'temp_c': 17.1, 'temp_f': 62.8, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 273, 'wind_dir': 'W', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 55, 'cloud': 0, 'feelslike_c': 17.1, 'feelslike_f': 62.8, 'windchill_c': 15.6, 'windchill_f': 60.1, 'heatindex_c': 15.4, 'heatindex_f': 59.7, 'dewpoint_c': 11.3, 'dewpoint_f': 52.4, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 9.9, 'gust_kph': 15.9}}"}, {'url': 'https://www.timeanddate.com/weather/@z-us-94

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [7]:
model_with_tools = model.bind_tools(tools)


In [8]:
response = model_with_tools.invoke([HumanMessage(content="hi!")])
print("Content:", response.content)
print("Tool Calls:", response.tool_calls)



Content: Hello! How can I assist you today?
Tool Calls: []


In [9]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_eu48YNP1ZZuHWEVgmhhVLqNU', 'type': 'tool_call'}]


## Create an agent

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='d41f41a3-b9d4-4714-8940-16ec5847c720'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8552ec53e1', 'finish_reason': 'stop', 'logprobs': None}, id='run-73679294-f913-4a1d-96d1-3598057a9003-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

In [12]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='dcf5166a-c2de-4b36-a084-da6665559d51'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h9D9qnEQYWE5SBwyUaXLBfTz', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e8ad56ff-4f96-446d-a75d-9177dcf219ee-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_h9D9qnEQYWE5SBwyUaXLBfTz', 'type': 'tool_call'}], usage_metadata={'i

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zq95DXnMFNI8xLazaTCJGJMc', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f654efe7-8927-44bc-8917-939b5fd93e9f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_zq95DXnMFNI8xLazaTCJGJMc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108, 'input_token_details': {'cache_read': 0}, 'output_token

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [ ]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatOpenAI(model_name="gpt-4o-mini")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")